<header style="background-image: url('img/metropolis-tv-series-pic.jpg'); background-size: cover; padding: 50px 0;font-size: 1.5em;background-repeat: no-repeat;">
  <div style="padding: 10px 25px; background-color: #fff9 !important;">
  <h1 style="color: black;text-shadow: #999 5px 5px 0px; margin-bottom: 2em">Is it a bird?<br/>Is it a plane?<br/>Accelerating Python with numba</h1>
  <h2 style="text-align: right;font-size: 1em;">Juan Luis Cano Rodríguez &lt;hello@juanlu.space&gt;<br>2020-02-28 @ Python Majorca</h2>
  </div>
  </header>

# Outline

1. "Python is slow"
2. What is numba?
3. Some examples
4. Limitations and workarounds
5. Conclusions

# Who is this guy?

* **Aerospace Engineer** with a passion for orbits 🛰
* Former chair of the **Python España** non profit and former co-organizer of **PyCon Spain** 🐍
* **Software Developer** at **Satellogic** 🌍
* Free Software advocate and Python enthusiast 🕮
* Hard Rock lover 🎸

Follow me! https://github.com/Juanlu001/

![Me!](img/juanlu_esa.jpg)

# "Python is slow"

## Data structures

![array vs list](img/array_vs_list.png)

_(From https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/_)

## Dynamic and interpreted (rather than static and compiled)

![Four nested loops](img/loops.png)

_(From https://gist.github.com/Juanlu001/cf19b1c16caf618860fb_)

## Introspection

!["Why is checking isinstance(something, Mapping) so slow?"](img/isinstance.png)

_(From https://stackoverflow.com/q/42378726/554319)_

## What to do?

### Vectorization

* Rewriting some code leveraging high level NumPy functions can make it way faster
* However, this works best for array manipulation - some other algorithms cannot easily be vectorized
* And even if you can, vectorized code can be impossible to read

![Too smart](img/too_smart.png)

### Cython

![Cython](img/Cython-logo.png)

* Mature, widely used, effective, gradual - a great project!
* Some personal problems with it:
  - I don't know any C, so it's more difficult for me
  - I wanted poliastro to be super easy to install by avoiding the "two language" problem (this includes Windows)
  - The native debugger is broken https://github.com/Juanlu001/cython-rasterio-debugging/issues/2
  - I really don't want to worry about some gore details

I don't have lots of experience with it, so I don't have solid arguments against it.

### PyPy

![PyPy](img/pypy-logo.png)

* PyPy is a super interesting alternative Python implementation https://pypy.org/
* I really really want to use it more, but there are some obstacles:
  - The documentation is a bit poor, even the changelogs
  - Lacks interest from the mainstream community (including snarky comments by Guido about "nobody using it in production")
  - Support in conda is half-broken https://github.com/conda-forge/pypy2.7-feedstock/issues/1
  - PyPy has several incompatibilities with manylinux1 wheels https://bitbucket.org/pypy/pypy/issues/2617/
  - manylinux2010 are almost there, but need the final push https://github.com/pypa/manylinux/issues/179

### And a lot others...

https://github.com/pfalcon/awesome-python-compilers/

The number of projects keeps growing

## All of a sudden, in 2012...

![numba 0.1 is released](img/tweet-travis.png)

https://twitter.com/teoliphant/status/235789560678858752

# What is numba?

![numba](img/numba.png)

> Numba is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code.

* Latest version (at the time of writing) 0.48, 10 days ago
* Documentation https://numba.pydata.org/numba-doc/latest/index.html
* BSD-2 License
* Easy to install:

```
$ pip install numba
$ conda install numba [--channel conda-forge]
```

## Example 1: Monte Carlo for $\pi$

![Monte Carlo pi](img/Pi_30K.gif)

(From https://commons.wikimedia.org/wiki/File:Pi_30K.gif)

In [1]:
import random

def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [2]:
monte_carlo_pi(100)

3.16

In [3]:
monte_carlo_pi(100_000)

3.13924

In [4]:
%timeit monte_carlo_pi(10_000_000)  # Slow!

4.25 s ± 228 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
from numba import jit

In [6]:
monte_carlo_pi_fast = jit(monte_carlo_pi)

In [7]:
%timeit -n1 -r1 monte_carlo_pi_fast(100)

338 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [8]:
%timeit monte_carlo_pi_fast(10_000_000)  # 40x faster!

121 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Example 2: Plate deflection

$$
w(\xi, \eta) = \frac{4 P_c}{\pi^4 D L_x L_y} \sum_{m=1}^\infty \sum_{n=1}^\infty \frac{\sin{\frac{m \pi \xi}{L_x}} \sin \frac{n \pi \eta}{L_y}}{\left(\left(\frac{m}{L_x}\right)^2 + \left(\frac{n}{L_y}\right)^2\right)^2}
$$

(From http://www.efunda.com/formulae/solid_mechanics/plates/calculators/SSSS_PPoint.cfm)

In [9]:
import numpy as np
from numpy import pi, sin

In [10]:
@jit
def a_mn_point(P, a, b, xi, eta, mm, nn):
    """Navier series coefficient for concentrated load."""
    return 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)

In [11]:
@jit
def plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    max_i, max_j = ww.shape
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = a_mn_point(P, a, b, xi, eta, mm, nn)
                    ww[ii, jj] += (
                        a_mn
                        / (mm ** 2 / a ** 2 + nn ** 2 / b ** 2) ** 2
                        * sin(mm * pi * xx[ii, jj] / a)
                        * sin(nn * pi * yy[ii, jj] / b)
                        / (pi ** 4 * D)
                    )

In [12]:
# Plate geometry
a = 1.0  # m
b = 1.0  # m
h = 50e-3  # m

# Material properties
E = 69e9  # Pa
nu = 0.35

# Series terms
max_m = 16
max_n = 16

# Computation points
# NOTE: With an odd number of points the center of the place is included in
# the grid
NUM_POINTS = 101

# Load
P = -10e3  # N
xi = a / 2
eta = a / 2

# Flexural rigidity
D = h**3 * E / (12 * (1 - nu**2))

# Set up domain
x = np.linspace(0, a, num=NUM_POINTS)
y = np.linspace(0, b, num=NUM_POINTS)
xx, yy = np.meshgrid(x, y)

In [13]:
# Compute displacement field
ww = np.zeros_like(xx)
%timeit plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

111 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit -n1 -r1 plate_displacement.py_func(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)  # 100 times slower!

17.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
# https://github.com/plotly/plotly.py/issues/1664#issuecomment-511773518
import plotly.graph_objects as go
import plotly.io as pio

# Set default renderer
pio.renderers.default = "notebook_connected"

# Set default template
pio.templates["slides"] = go.layout.Template(layout=dict(width=800, height=550))
pio.templates.default = "plotly+slides"

In [16]:
fig = go.Figure()
fig.add_surface(z=ww)

## Example 3: A ~~shameless self-plug~~ personal achievement

"poliastro: An Astrodynamics library written in Python with Fortran performance" at the 6th International Conference on Astrodynamics Tools and Techniques

![poliastro benchmark](img/poliastro-benchmark.png)

https://indico.esa.int/event/111/contributions/393/

## Example 4: C extensions with CFFI!

We can call C functions exported using CFFI:

https://web.archive.org/web/20160611082327/https://www.continuum.io/blog/developer-blog/calling-c-libraries-numba-using-cffi

```python
from numba import njit, cffi_support

# See https://www.pybonacci.org/2016/02/07/como-crear-extensiones-en-c-para-python-usando-cffi-y-numba/
# (Spanish, sorry!)
import _hyper
cffi_support.register_module(_hyper)

_hyp2f1 = _hyper.lib.hyp2f1x  # See https://github.com/numba/numba/issues/1688


@njit
def hyp2f1(a, b, c, x):
    return _hyp2f1(a, b, c, x)
```

# Caveats and limitations ☹️

![python-sgp4 fail](img/python-sgp4-fail1.png)

![python-sgp4 fail](img/python-sgp4-fail2.png)

## The "nopython mode" is the only way

* Two modes: "object mode" and "nopython mode", only the latter is truly optimized
* Functions JITted in nopython mode can only call other functions in nopython mode
* _Avoid "object mode"!_ In the process of being deprecated, in numba 0.44 raises warnings

![It's nopython all the way down](img/nopython.jpg)

In [17]:
@jit
def range10():
    l = []
    for x in range(10):
        l.append(x)
    return l

range10()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [18]:
@jit
def reversed_range10():
    l = []
    for x in range(10):
        l.append(x)

    return reversed(l)  # innocuous change, but no reversed support in nopython mode

reversed_range10()

<ipython-input-18-2e1fa93510fc>:1: NumbaWarning:


Compilation is falling back to object mode WITH looplifting enabled because Function "reversed_range10" failed type inference due to: Untyped global name 'reversed': cannot determine Numba type of <class 'type'>

File "<ipython-input-18-2e1fa93510fc>", line 7:
def reversed_range10():
    <source elided>

    return reversed(l)  # innocuous change, but no reversed support in nopython mode
    ^


<ipython-input-18-2e1fa93510fc>:1: NumbaWarning:


Compilation is falling back to object mode WITHOUT looplifting enabled because Function "reversed_range10" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-18-2e1fa93510fc>", line 4:
def reversed_range10():
    <source elided>
    l = []
    for x in range(10):
    ^


/home/juanlu/.pyenv/versions/3.8.0/envs/numba38/lib/python3.8/site-packages/numba/object_mode_passes.py:177: NumbaWarning:

Function "reversed_range10" was c

In [19]:
@jit(nopython=True)
def reversed_range10():
    l = []
    for x in range(10):
        l.append(x)

    return l[::-1]  # innocuous change, but no reversed support in nopython mode

reversed_range10()

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [20]:
from numba import njit

@njit
def reversed_range10():
    l = []
    for x in range(10):
        l.append(x)

    return l[::-1]  # innocuous change, but no reversed support in nopython mode

reversed_range10()

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

## Passing functions as arguments is _slow_

* Since numba 0.38 the user can pass JITted functions as arguments, but it's even slower than not JITting them https://github.com/numba/numba/issues/2952
* Arguably the most important blocker to write reusable numba code

In [21]:
@njit
def func(x):
    return x**3 - 1

@njit
def fprime(x):
    return 3 * x**2

In [22]:
@njit
def njit_newton(func, x0, fprime):
    for _ in range(50):
        fder = fprime(x0)
        fval = func(x0)
        newton_step = fval / fder
        x = x0 - newton_step
        if abs(x - x0) < 1.48e-8:
            return x
        x0 = x

In [23]:
%timeit njit_newton(func, 1.5, fprime)
%timeit njit_newton.py_func(func, 1.5, fprime=fprime)

21 µs ± 687 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.46 µs ± 58 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


With a smart combination of closures and caching we can implement a workaround:

In [24]:
from functools import lru_cache

In [25]:
@lru_cache()
def newton_generator(func, fprime):
    @njit
    def njit_newton_final(x0):
        for _ in range(50):
            fder = fprime(x0)
            fval = func(x0)
            newton_step = fval / fder
            x = x0 - newton_step
            if abs(x - x0) < 1.48e-8:
                return x
            x0 = x

    return njit_newton_final

In [26]:
%timeit -n1 -r1 newton_generator(func, fprime)

250 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [27]:
%timeit -n1 -r1 newton_generator(func, fprime)

2.71 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [28]:
newton_func = newton_generator(func, fprime)
%timeit -n1 -r1 newton_func(1.5)

80.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [29]:
%timeit newton_func(1.5)

227 ns ± 5.89 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [30]:
%timeit newton_generator(func, fprime)

173 ns ± 8.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [31]:
def newton(func, x0, fprime):
    return newton_generator(func, fprime)(x0)

%timeit newton(func, 1.5, fprime)

515 ns ± 55.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## NumPy arrays and nothing else

![Two layers](img/two-layers.png)

High level API:

* Supports complex data structures (e.g. `astropy.units` or `pint`, NumPy extensions for physical units) 
* Convert the code to normalized, simple structure that numba understands

Dangerous™ algorithms:

* Fast (easy to accelerate with `numba.njit`)
* Only cares about numbers, makes assumptions

## Tips and tricks

* If anything fails, `export NUMBA_DISABLE_JIT=1`
* Be careful with dtypes! https://github.com/numba/numba/issues/3993#issuecomment-485029668
* Try to split the function in smaller chunks until you find out what triggers object (slow) mode
* You might have to rewrite some stuff, make it less dynamic
* Keep an eye on https://numba.pydata.org/numba-doc/dev/reference/pysupported.html and https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

## Not covered in this talk

* Ahead-of-time (AOT) compilation
* Interface with GPUs (CUDA, ROCm)
* Releasing the GIL

Check out the documentation! https://numba.pydata.org/numba-doc/latest/

# Conclusions

* Numba is ✨ _awesome_ ✨ when you make it work!
* Still requires a bit of code rewrites, but the code is still _mostly pythonic_ Python
* For non-numerical code, you will probably have to find something else

# Per Python ad astra! 🚀

* Slides https://github.com/Juanlu001/talk-numba
* My email <hello@juanlu.space>
* My Twitter https://twitter.com/poliastro_py

![Vega launch](img/vega.jpg)

# Backup slides

## Comparison of solutions

| Project | Pros | Cons |
|--------|-----------------------------------------|-----------------------------------------------------------------------------------------------|
| NumPy | Powerful, omnipresent  | Vectorized code is sometimes difficult to read<sup>1</sup>, if you can't vectorize you are out of luck |
| Cython | Gradual, effective, widely used, mature | Tricky if you don't know any C, couldn't make the native debugger work<sup>2</sup> |
| PyPy | General purpose | C extensions still very slow, no wheels on PyPI |
| Numba | Simplest, very effective | Only numerical code, needs special care |

And many others: Pythran, Nuitka, mypyc...

<sup>1</sup>Check out "Integration with the vernacular", by James Powell https://pyvideo.org/pydata-london-2015/integration-with-the-vernacular.html

<sup>2</sup>https://github.com/cython/cython/issues/2699

<sup>3</sup>See https://github.com/antocuni/pypy-wheels for a half-baked effort. Perhaps the future will be brighter with the new manylinux2010 specification? https://bitbucket.org/pypy/pypy/issues/2617/pypy-binary-is-linked-to-too-much-stuff, https://github.com/pypa/manylinux/issues/179